# Selecting Data

## Introduction

As a data scientist, the SQL query you'll likely use most often is `SELECT`. This lesson introduces how to use `SELECT` to subset and transform the columns of a database table.

## Objectives

You will be able to:
- Retrieve a subset of columns from a table
- Use built-in SQL functions to transform selected columns
- Retrieve a subset of records from a table using a basic `WHERE` clause

## The Data

Below, we connect to a SQLite database using the Python `sqlite3` library ([documentation here](https://docs.python.org/3/library/sqlite3.html)):

In [1]:
import sqlite3 
conn = sqlite3.connect('data.sqlite')

The database that you've just connected to is the same database you have seen previously, containing data about orders, employeers, etc. Here's an overview of the database:  

<img src="images/Database-Schema.png">

For this first section we'll be focusing on the `employees` table.

If we want to get all information about the employee records, we might do something like this (`*` means all columns):

In [2]:
import pandas as pd
pd.read_sql("""SELECT * FROM employees;""", conn)

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,,President
1,1056,Patterson,Mary,x4611,mpatterso@classicmodelcars.com,1,1002,VP Sales
2,1076,Firrelli,Jeff,x9273,jfirrelli@classicmodelcars.com,1,1002,VP Marketing
3,1088,Patterson,William,x4871,wpatterson@classicmodelcars.com,6,1056,Sales Manager (APAC)
4,1102,Bondur,Gerard,x5408,gbondur@classicmodelcars.com,4,1056,Sale Manager (EMEA)
5,1143,Bow,Anthony,x5428,abow@classicmodelcars.com,1,1056,Sales Manager (NA)
6,1165,Jennings,Leslie,x3291,ljennings@classicmodelcars.com,1,1143,Sales Rep
7,1166,Thompson,Leslie,x4065,lthompson@classicmodelcars.com,1,1143,Sales Rep
8,1188,Firrelli,Julie,x2173,jfirrelli@classicmodelcars.com,2,1143,Sales Rep
9,1216,Patterson,Steve,x4334,spatterson@classicmodelcars.com,2,1143,Sales Rep


### Quick Note on String Syntax

When working with strings, you may have previously seen a `'string'`, a `"string"`, a `'''string'''`, or a `"""string"""`. While all of these are strings, the triple quotes have the added functionality of being able to use multiple lines within the same string as well as to use single quotes within the string. Sometimes, SQL queries can be much longer than others, in which case it's helpful to use new lines for readability. Here's the same example, this time with the string spread out onto multiple lines:

In [3]:
pd.read_sql("""
SELECT *
  FROM employees;
""", conn)

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,,President
1,1056,Patterson,Mary,x4611,mpatterso@classicmodelcars.com,1,1002,VP Sales
2,1076,Firrelli,Jeff,x9273,jfirrelli@classicmodelcars.com,1,1002,VP Marketing
3,1088,Patterson,William,x4871,wpatterson@classicmodelcars.com,6,1056,Sales Manager (APAC)
4,1102,Bondur,Gerard,x5408,gbondur@classicmodelcars.com,4,1056,Sale Manager (EMEA)
5,1143,Bow,Anthony,x5428,abow@classicmodelcars.com,1,1056,Sales Manager (NA)
6,1165,Jennings,Leslie,x3291,ljennings@classicmodelcars.com,1,1143,Sales Rep
7,1166,Thompson,Leslie,x4065,lthompson@classicmodelcars.com,1,1143,Sales Rep
8,1188,Firrelli,Julie,x2173,jfirrelli@classicmodelcars.com,2,1143,Sales Rep
9,1216,Patterson,Steve,x4334,spatterson@classicmodelcars.com,2,1143,Sales Rep


Unlike in Python, whitespace indentation in SQL is not used to indicate scope or any other important information. Therefore this:

```sql
SELECT *
  FROM employees;
```

(with two spaces in front of `FROM`)

is identical to this:

```sql
SELECT *
FROM employees;
```

(with zero spaces in front of `FROM`)

as far as SQL is concerned. However we will be aligning the right edge of the SQL keywords, using a "river" of whitespace down the center to improve legibility in this lesson, following [this style guide](https://www.sqlstyle.guide/). You will see multi-line SQL written with various different indentation styles, and you will want to check with your employer to learn what their style guide is.

## Retrieving a Subset of Columns

Once we know what the column names are for a given table, we can select specific columns rather than using `*` to select all of them. This is achieved by replacing the `*` with the names of the columns, separated by commas.

For example, if we just wanted to select the last and first names of the employees:

In [4]:
pd.read_sql("""
SELECT lastName, firstName
  FROM employees;
""", conn).head()

,lastName,firstName
0,Murphy,Diane
1,Patterson,Mary
2,Firrelli,Jeff
3,Patterson,William
4,Bondur,Gerard


We can also specify the columns in a different order than they appear in the database, in order to reorder the columns in the resulting dataframe:

In [5]:
pd.read_sql("""
SELECT firstName, lastName
  FROM employees;
""", conn).head()

,firstName,lastName
0,Diane,Murphy
1,Mary,Patterson
2,Jeff,Firrelli
3,William,Patterson
4,Gerard,Bondur


Additionally, we can use aliases (`AS` keyword) to change the column names in our query result:

In [6]:
pd.read_sql("""
SELECT firstName AS name
  FROM employees;
""", conn).head()

,name
0,Diane
1,Mary
2,Jeff
3,William
4,Gerard


## Using Built-in SQL Functions

Similar to the [Python built-in functions](https://docs.python.org/3/library/functions.html), SQL also has built-in functions. The available functions will differ somewhat by the type of SQL you are using, but in general you should be able to find functions for:

* String manipulation
* Math operations
* Date and time operations

For SQLite in particular, if you are looking for a built-in function, start by checking the [core functions](https://www.sqlite.org/lang_corefunc.html) page, [mathematical functions](https://www.sqlite.org/lang_mathfunc.html) page, and/or [date and time functions](https://www.sqlite.org/lang_datefunc.html) page.

### Built-in SQL Functions for String Manipulation

#### `length`

Let's start with an example of a SQL built-in function that is very similar to one we have in Python: `length` ([documentation here](https://www.sqlite.org/lang_corefunc.html#length)). This works very similarly to the `len` built-in function in Python. For a string, it returns the number of characters.

If we wanted to find the length of the first names of all employees, that would look like this:

In [7]:
pd.read_sql("""
SELECT length(firstName) AS name_length
  FROM employees;
""", conn).head()

,name_length
0,5
1,4
2,4
3,7
4,6


#### `upper`

Now let's say we wanted to return all of the employee names in all caps. Similar to the Python string method, this SQL function is called `upper` ([documentation here](https://www.sqlite.org/lang_corefunc.html#upper)). However, since it's a built-in function and not a method, the syntax looks like:

```sql
upper(column_name)
```

and not `column_name.upper()`.

As you get more comfortable with Python and SQL, distinctions like this will get more intuitive, but for now don't worry if you have to look it up every time!

Here is an example using `upper`:

In [8]:
pd.read_sql("""
SELECT upper(firstName) AS name_in_all_caps
  FROM employees;
""", conn).head()

,name_in_all_caps
0,DIANE
1,MARY
2,JEFF
3,WILLIAM
4,GERARD


#### `substr`

Another form of string manipulation you might need is finding a substring (subset of a string). In Python, we do this with string slicing. In SQL, there is a built-in function that does this instead. For SQLite specifically, this is called `substr` ([documentation here](https://www.sqlite.org/lang_corefunc.html#substr)).

Let's say we wanted just the first initial (first letter of the first name) for each employee:

In [9]:
pd.read_sql("""
SELECT substr(firstName, 1, 1) AS first_initial
  FROM employees;
""", conn).head()

,first_initial
0,D
1,M
2,J
3,W
4,G


If we wanted to add a `.` after each first initial, we could use the SQLite `||` (concatenate) operator. This works similarly to `+` with strings in Python:

In [10]:
pd.read_sql("""
SELECT substr(firstName, 1, 1) || "." AS first_initial
  FROM employees;
""", conn).head()

,first_initial
0,D.
1,M.
2,J.
3,W.
4,G.


We can also combine multiple column values, not just string literals. For example:

In [11]:
pd.read_sql("""
SELECT firstName || " " || lastName AS full_name
  FROM employees;
""", conn).head()

,full_name
0,Diane Murphy
1,Mary Patterson
2,Jeff Firrelli
3,William Patterson
4,Gerard Bondur


### Built-in SQL Functions for Math Operations

For these examples, let's switch over to using the `orderDetails` table:

In [12]:
pd.read_sql("""SELECT * FROM orderDetails;""", conn)

,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber
0,10100,S18_1749,30,136.00,3
1,10100,S18_2248,50,55.09,2
2,10100,S18_4409,22,75.46,4
3,10100,S24_3969,49,35.29,1
4,10101,S18_2325,25,108.06,4
...,...,...,...,...,...
2991,10425,S24_2300,49,127.79,9
2992,10425,S24_2840,31,31.82,5
2993,10425,S32_1268,41,83.79,11
2994,10425,S32_2509,11,50.32,6


#### `round`

Let's say we wanted to round the price to the nearest dollar. We could use the SQL `round` function ([documentation here](https://www.sqlite.org/lang_corefunc.html#round)), which is very similar to the the Python `round`:

In [13]:
pd.read_sql("""
SELECT round(priceEach) AS rounded_price
  FROM orderDetails;
""", conn)

,rounded_price
0,136.0
1,55.0
2,75.0
3,35.0
4,108.0
...,...
2991,128.0
2992,32.0
2993,84.0
2994,50.0


#### `CAST`

The previous result looks ok, but it's returning floating point numbers. What if we want integers instead?

In Python, we might apply the `int` built-in function. In SQLite, we can use a `CAST` expression ([documentation here](https://www.sqlite.org/lang_expr.html#castexpr)):

In [14]:
pd.read_sql("""
SELECT CAST(round(priceEach) AS INTEGER) AS rounded_price_int
  FROM orderDetails;
""", conn)

,rounded_price_int
0,136
1,55
2,75
3,35
4,108
...,...
2991,128
2992,32
2993,84
2994,50


#### Basic Math Operations

Just like when performing math operations with Python, you don't always need to use a function. Sometimes all you need is an operator like `+`, `-`, `/`, or `*`. For example, below we multiply the price times the quantity ordered to find the total price:

In [15]:
pd.read_sql("""
SELECT priceEach * quantityOrdered AS total_price
  FROM orderDetails;
""", conn)

,total_price
0,4080.00
1,2754.50
2,1660.12
3,1729.21
4,2701.50
...,...
2991,6261.71
2992,986.42
2993,3435.39
2994,553.52


### Built-in SQL Functions for Date and Time Operations

For these examples, we'll look at yet another table within the database, this time the `orders` table:

In [16]:
pd.read_sql("""SELECT * FROM orders;""", conn)

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,,363
1,10101,2003-01-09,2003-01-18,2003-01-11,Shipped,Check on availability.,128
2,10102,2003-01-10,2003-01-18,2003-01-14,Shipped,,181
3,10103,2003-01-29,2003-02-07,2003-02-02,Shipped,,121
4,10104,2003-01-31,2003-02-09,2003-02-01,Shipped,,141
...,...,...,...,...,...,...,...
321,10421,2005-05-29,2005-06-06,,In Process,Custom shipping instructions were sent to ware...,124
322,10422,2005-05-30,2005-06-11,,In Process,,157
323,10423,2005-05-30,2005-06-05,,In Process,,314
324,10424,2005-05-31,2005-06-08,,In Process,,141


What if we wanted to know how many days there are between the `requiredDate` and the `orderDate` for each order? Intuitively you might try something like this:

In [17]:
pd.read_sql("""
SELECT requiredDate - orderDate
  FROM orders;
""", conn)

,requiredDate - orderDate
0,0
1,0
2,0
3,0
4,0
...,...
321,0
322,0
323,0
324,0


Clearly that didn't work.

It turns out that we need to specify that we want the difference in *days*. One way to do this is using the `julianday` function ([documentation here](https://www.sqlite.org/lang_datefunc.html)):

In [18]:
pd.read_sql("""
SELECT julianday(requiredDate) - julianday(orderDate) AS days_from_order_to_required
  FROM orders;
""", conn)

,days_from_order_to_required
0,7.0
1,9.0
2,8.0
3,9.0
4,9.0
...,...
321,8.0
322,12.0
323,6.0
324,8.0


If we wanted to select the dates order dates as well as 1 week after the order dates, that would look like this:

In [19]:
pd.read_sql("""
SELECT orderDate, date(orderDate, "+7 days") AS one_week_later
  FROM orders;
""", conn)

,orderDate,one_week_later
0,2003-01-06,2003-01-13
1,2003-01-09,2003-01-16
2,2003-01-10,2003-01-17
3,2003-01-29,2003-02-05
4,2003-01-31,2003-02-07
...,...,...
321,2005-05-29,2005-06-05
322,2005-05-30,2005-06-06
323,2005-05-30,2005-06-06
324,2005-05-31,2005-06-07


You can also use the `strftime` function, which is very similar to the [Python version](https://docs.python.org/3/library/datetime.html#strftime-strptime-behavior). This is useful if you want to split apart a date or time value into different sub-parts. For example, here we extract the year, month, and day of month:

In [20]:
pd.read_sql("""
SELECT orderDate,
       strftime("%m", orderDate) AS month,
       strftime("%Y", orderDate) AS year,
       strftime("%d", orderDate) AS day
  FROM orders;
""", conn)

,orderDate,month,year,day
0,2003-01-06,01,2003,06
1,2003-01-09,01,2003,09
2,2003-01-10,01,2003,10
3,2003-01-29,01,2003,29
4,2003-01-31,01,2003,31
...,...,...,...,...
321,2005-05-29,05,2005,29
322,2005-05-30,05,2005,30
323,2005-05-30,05,2005,30
324,2005-05-31,05,2005,31


## Introduction to the `WHERE` Clause

With just a `SELECT` expression, we can specify which **columns** we want to select, as well as transform the column values using aliases, built-in functions, and other expressions.

However if we want to filter the **rows** that we want to select, we also need to include a `WHERE` clause. Below are a few examples of the `SELECT` queries from above, with `WHERE` clauses added.

### Selecting Employees Based on String Conditions

If we wanted to select all employees with 5 letters in their first name, that would look like this:

In [21]:
pd.read_sql("""
SELECT *, length(firstName) AS name_length
  FROM employees
 WHERE name_length = 5;
""", conn)

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,name_length
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,,President,5
1,1188,Firrelli,Julie,x2173,jfirrelli@classicmodelcars.com,2,1143,Sales Rep,5
2,1216,Patterson,Steve,x4334,spatterson@classicmodelcars.com,2,1143,Sales Rep,5
3,1501,Bott,Larry,x2311,lbott@classicmodelcars.com,7,1102,Sales Rep,5
4,1504,Jones,Barry,x102,bjones@classicmodelcars.com,7,1102,Sales Rep,5
5,1612,Marsh,Peter,x102,pmarsh@classicmodelcars.com,6,1088,Sales Rep,5


Or, to select all employees with the first initial of "L.", that would look like this:

In [22]:
pd.read_sql("""
SELECT *, substr(firstName, 1, 1) AS first_initial
  FROM employees
 WHERE first_initial = "L";
""", conn)

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,first_initial
0,1165,Jennings,Leslie,x3291,ljennings@classicmodelcars.com,1,1143,Sales Rep,L
1,1166,Thompson,Leslie,x4065,lthompson@classicmodelcars.com,1,1143,Sales Rep,L
2,1337,Bondur,Loui,x6493,lbondur@classicmodelcars.com,4,1102,Sales Rep,L
3,1501,Bott,Larry,x2311,lbott@classicmodelcars.com,7,1102,Sales Rep,L


**Important note:** Just like in Python, you can compare numbers in SQL just by typing the number (e.g. `name_length = 5`) but if you want to compare to a string value, you need to surround the value with quotes (e.g. `first_initial = "L"`). If you forget the quotes, you will get an error, because SQL will interpret it as a variable name rather than a hard-coded value:

In [23]:
pd.read_sql("""
SELECT *, substr(firstName, 1, 1) AS first_initial
  FROM employees
 WHERE first_initial = L;
""", conn)

DatabaseError: Execution failed on sql '
SELECT *, substr(firstName, 1, 1) AS first_initial
  FROM employees
 WHERE first_initial = L;
': no such column: L

### Selecting Order Details Based on Price

Below we select all order details where the price each, rounded to the nearest integer, is 30 dollars:

In [24]:
pd.read_sql("""
SELECT *, CAST(round(priceEach) AS INTEGER) AS rounded_price_int
  FROM orderDetails
 WHERE rounded_price_int = 30;
""", conn)

,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber,rounded_price_int
0,10104,S24_2840,44,30.41,10,30
1,10173,S24_1937,31,29.87,9,30
2,10184,S24_2840,42,30.06,7,30
3,10280,S24_1937,20,29.87,12,30
4,10332,S24_1937,45,29.87,6,30
5,10367,S24_1937,23,29.54,13,30
6,10380,S24_1937,32,29.87,4,30


### Selecting Orders Based on Date

We can use the `strftime` function to select all orders placed in January of any year:

In [25]:
pd.read_sql("""
SELECT *, strftime("%m", orderDate) AS month
  FROM orders
 WHERE month = "01";
""", conn)

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber,month
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,,363,01
1,10101,2003-01-09,2003-01-18,2003-01-11,Shipped,Check on availability.,128,01
2,10102,2003-01-10,2003-01-18,2003-01-14,Shipped,,181,01
3,10103,2003-01-29,2003-02-07,2003-02-02,Shipped,,121,01
4,10104,2003-01-31,2003-02-09,2003-02-01,Shipped,,141,01
5,10208,2004-01-02,2004-01-11,2004-01-04,Shipped,,146,01
6,10209,2004-01-09,2004-01-15,2004-01-12,Shipped,,347,01
7,10210,2004-01-12,2004-01-22,2004-01-20,Shipped,,177,01
8,10211,2004-01-15,2004-01-25,2004-01-18,Shipped,,406,01
9,10212,2004-01-16,2004-01-24,2004-01-18,Shipped,,141,01


We can also check to see if any orders were shipped late (`shippedDate` after `requiredDate`, i.e. the number of days late is a positive number):

In [26]:
pd.read_sql("""
SELECT *, julianday(shippedDate) - julianday(requiredDate) AS days_late
  FROM orders
 WHERE sign(days_late) = +1;
""", conn)

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber,days_late
0,10165,2003-10-22,2003-10-31,2003-12-26,Shipped,This order was on hold because customers's cre...,148,56.0


Now that we are finished with our queries, we can close the database connection.

In [27]:
conn.close()

## Summary

In this lesson, you saw how to execute SQL `SELECT` queries, specifying which columns to be selected as well as transforming the columns using string, numeric, and date/time built-in functions and expressions. You also saw how to filter the resulting rows using a `WHERE` clause that checked whether a given column was equal to a specific value. Going forward, you'll learn more sophisticated techniques using the `WHERE` clause as well as additional keywords for specifying your query parameters!